In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "omriabo/dictalm_squad_finetuned_test"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

The repository for dicta-il/dictalm-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/dicta-il/dictalm-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_megatron_gpt.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dicta-il/dictalm-7b:
- configuration_megatron_gpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

In [3]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 0 | total: 5474381824 | Percentage: 0.0000%


In [9]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  ### הקשר: {query}
  ### שאלה:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [5]:
model.half()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MegatronGPTForCausalLM(
      (megatron_gpt): MegatronGPTModel(
        (embed_in): Embedding(56064, 4096)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x MegatronGPTLayer(
            (input_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (self_attention): MegatronGPTAttention(
              (rotary_emb): MegatronGPTRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
   

In [7]:
result = get_completion(query="בקפריסין, יש עץ זית המכונה עץ זית וואסיליקוס, שמתויחסים אליו כאחד העצים העתיקים בעולם. מעריכים שהוא בן יותר מ-2,000 שנה. העץ נמצא בכפר לאטסי והוא נחשב לאטרקציה תיירותית מרכזית. מעניין לציין שעל אף גילו המופלג, העץ עדיין מניב זיתים, אשר משמשים ליצירת שמן זית מסורתי באזור.", model=model, tokenizer=tokenizer)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')



  ### הקשר: בקפריסין, יש עץ זית המכונה עץ זית וואסיליקוס, שמתויחסים אליו כאחד העצים העתיקים בעולם. מעריכים שהוא בן יותר מ-2,000 שנה. העץ נמצא בכפר לאטסי והוא נחשב לאטרקציה תיירותית מרכזית. מעניין לציין שעל אף גילו המופלג, העץ עדיין מניב זיתים, אשר משמשים ליצירת שמן זית מסורתי באזור.
  ### שאלה:
  ### שאלה:
מה קרה לעץ של זיתים בסיליקו?
### שאלה:
מה קרה לעץ של זיתים בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
למה העץ של הזיתים בסיליקו, פרט?
### שאלה:
מה קרה לעץ הזיתים בסיליקו?
### שאלה:
מה קרה לעץ הזיתים בסיליקו?
### שאלה:
מה קרה לעץ הזיתים בסיליקו?
### שאלה:
מה קרה לעץ הזית בסיליקו?
### שאלה:
מה היה לעץ של זית בסיליקו?
### שאלה:
מה היה לעץ הזית בסיליקו?
### שאלה:
איזו שאלה?
### שאלה:
מי היה לעץ של זית בסיליקויקויקונו?
### שאלה:
מה היה לעץ הזית בסיליקונו?
### שאלה:
מה היה לעץ הזית בסיליקויקויקווקוויניקו?
### שאלה:
מה היה לעץ של זיתים בסילי

In [8]:
result = get_completion(query="בעיר ניו יורק, באחת התחנות התת-קרקעיות של הרכבת, נמצא מה שנחשב לפסל הפיל הכי קטן בעולם. הפסל, המכונה הפיל של תחנת 14, ממוקם בתחנת הרכבת ברחוב 14 וחמישית, נעשה בשנת 1983 על ידי האמן טום אוטרנס. הפסל הזעיר, שגודלו כאצבע, נועד להיות גילוי של אמנות פתאומית ומשעשעת במרחב העירוני. הוא נחשב לאחת מהפינות הסודיות והמיוחדות של העיר.", model=model, tokenizer=tokenizer)
print(result)


  ### הקשר: בעיר ניו יורק, באחת התחנות התת-קרקעיות של הרכבת, נמצא מה שנחשב לפסל הפיל הכי קטן בעולם. הפסל, המכונה הפיל של תחנת 14, ממוקם בתחנת הרכבת ברחוב 14 וחמישית, נעשה בשנת 1983 על ידי האמן טום אוטרנס. הפסל הזעיר, שגודלו כאצבע, נועד להיות גילוי של אמנות פתאומית ומשעשעת במרחב העירוני. הוא נחשב לאחת מהפינות הסודיות והמיוחדות של העיר.
  ### שאלה:
  ### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן בעולם?
### שאלה:
מה הוא פסל הפיל הכי קטן

In [10]:
result = get_completion(query="במדבר גובי, נמצאת תופעה טבעית ייחודית המכונה הזמרת של הדיונות. כאשר הרוח נושבת על הדיונות החוליות, הן מייצרות צלילים הדומים לזמירה או תזמורת. הצלילים הללו נוצרים עקב החדרת חול דק בין שכבות החול הגס יותר, תופעה שמתרחשת רק בתנאים מאוד מסוימים. התופעה הזו היא נדירה מאוד וניתן למצוא אותה רק במספר מקומות בעולם, והיא מושכת תיירים וחוקרים מרחבי העולם.", model=model, tokenizer=tokenizer)
print(result)


  ### הקשר: במדבר גובי, נמצאת תופעה טבעית ייחודית המכונה הזמרת של הדיונות. כאשר הרוח נושבת על הדיונות החוליות, הן מייצרות צלילים הדומים לזמירה או תזמורת. הצלילים הללו נוצרים עקב החדרת חול דק בין שכבות החול הגס יותר, תופעה שמתרחשת רק בתנאים מאוד מסוימים. התופעה הזו היא נדירה מאוד וניתן למצוא אותה רק במספר מקומות בעולם, והיא מושכת תיירים וחוקרים מרחבי העולם.
  ### שאלה:
  ### שאלה:
כיצד נוצרת תופעת הטבע הייחודית "הזמרת הדיונות"?
### שאלה:
### שאלה:
למה הן נוצרות?
### שאלה:
למה הן מושפעות?
### שאלה:
מתי התרחשה תופעת התרבות המוזרה הזו?
### שאלה:
מי מפיק את התופעה הזו ומה גורם ליצירת התופעה?
### שאלה:
מה מפיק את ה"התמרת הטבע" הזאת?
### שאלה
